In [25]:
from langchain_groq import ChatGroq
from typing import TypedDict
from typing_extensions import Annotated
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.graph import StateGraph, START, END
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

In [9]:
llm = ChatGroq(model = "llama-3.3-70b-versatile", groq_api_key = "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")

In [10]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [12]:
tool = TavilySearchResults(max_results = 2, tavily_api_key = "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")

In [14]:
tool.invoke("What is langgraph?")

[{'title': 'LangGraph - Overview - Docs by LangChain',
  'url': 'https://docs.langchain.com/oss/python/langgraph/overview',
  'content': 'Trusted by companies shaping the future of agents - including Klarna, Replit, Elastic, and more - LangGraph is a low-level orchestration framework for building, managing, and deploying long-running, stateful agents.',
  'score': 0.9405774},
 {'title': 'What is LangGraph? - Analytics Vidhya',
  'url': 'https://www.analyticsvidhya.com/blog/2024/07/langgraph-revolutionizing-ai-agent/',
  'content': 'To sum up, LangGraph is a major advancement in the development of AI agents. It enables developers to push the limits of what’s possible with AI agents by eliminating the shortcomings of earlier systems and offering a flexible, graph-based framework for agent construction and execution. LangGraph is positioned to influence the direction of artificial intelligence significantly in the future. [...] LangGraph is a library built on top of Langchain that is desi

In [30]:
def multiply(a: int, b: int)->int:
    """
    Multiply a and b

    Args:
        a (int): first int
        b (int): second int

    Returns:
        int: output int
    """
    return a * b

In [31]:
tools = [tool, multiply]

In [32]:
llm_with_tools = llm.bind_tools(tools)

In [33]:
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [40]:
builder_m = StateGraph(State)

In [41]:
builder_m.add_node("tool_calling_llm", tool_calling_llm)
builder_m.add_node("tools", ToolNode(tools))

builder_m.add_edge(START, "tool_calling_llm")
builder_m.add_conditional_edges("tool_calling_llm", tools_condition)
builder_m.add_edge("tools", "tool_calling_llm")

graph = builder_m.compile()

In [47]:
def get_response(query):
    response = graph.invoke({"messages": query})
    for m in response["messages"]:
        m.pretty_print()

In [48]:
print(get_response("What is recent ai news and 2 multiplied by 5"))

================================ Human Message =================================

What is recent ai news and 2 multiplied by 5
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (dc1sr9jpr)
 Call ID: dc1sr9jpr
  Args:
    query: recent ai news
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Latest AI News September 09 24 2025 | AI Morning Brief", "url": "https://techjacksolutions.com/ai-news-september-09-24-2025/", "content": "Three major developments reshaped the AI landscape yesterday. OpenAI’s massive infrastructure expansion gained momentum with five new data center sites worth $400 billion. Microsoft made a strategic pivot by integrating Anthropic’s Claude models into Copilot, diversifying beyond OpenAI for the first time in a major product. Healthcare got a practical AI breakthrough with a smart wound-healing device that actually wo